In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 데이터 수집 단계 함수
- [수정본]구석구석_크롤링_코드.ipynb
- 네이버_리뷰_크롤링_최종_배포용.ipynb

[수정본]구석구석_크롤링_코드.ipynb

In [ ]:
def button_click():
    # "인기순" 버튼 클릭
    popular_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "인기순")]')))
    driver.execute_script("arguments[0].click();", popular_button)

    # 서울 버튼 클릭
    seoul_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="1"]/button/span')))
    driver.execute_script("arguments[0].click();", seoul_button)

    # 서울 버튼 클릭 후, 해당 지역의 데이터가 로드될 때까지 대기
    wait = WebDriverWait(driver, 30)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/div[2]/div[1]/ul/li[1]/div[2]/div/a')))


# 각 페이지 데이터 추출 함수
# 이동한 페이지의 데이터를 추출하기 위해, 이동한 페이지의 html 소스를 가져와 파싱
def extract_data(soup, attraction_list, location_list, tag_list):
    for i in range(10):
        attraction = soup.select_one(f'#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child({i+1}) > div.area_txt.catchphrase > div > a')
        location = soup.select_one(f'#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child({i+1}) > div.area_txt.catchphrase > p:nth-child(2)')
        tag = soup.select_one(f'#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li:nth-child({i+1}) > div.area_txt.catchphrase > p.tag')

        # 명소, 위치, 해쉬 태그 3개를 다 가진 것만 추출
        if attraction and location and tag:
            attraction_list.append(attraction.text)
            location_list.append(location.text)
            tag_list.append(tag.text)

# 각 페이지 스크롤 내리는 함수
def slow_scroll_by(offset, duration):
    steps = 30  # 스크롤을 나누는 단계 수
    step_duration = duration / steps  # 각 단계별 대기 시간
    step_size = offset / steps  # 각 단계별 스크롤 크기
    for _ in range(steps):
        driver.execute_script(f"window.scrollBy(0, {step_size})") # steps 동안 step_size 만큼 반복해서 스크롤
        time.sleep(step_duration) # 1회 스크롤 하고 step_duration 만큼 대기

# 다음 페이지로 넘어가는 함수
def next_page(page_number):
    # 페이지 바에서 모든 페이지 링크 요소를 찾음
    page_bar = driver.find_elements(By.CSS_SELECTOR, '.page_box a')

    # 찾은 페이지 링크 요소들 중에서 지정된 페이지 번호(page_number)와 일치하는 버튼을 클릭
    for button in page_bar:
        if button.get_attribute("id") == str(page_number):
            # element.click() 기본 클릭 메소드
            # 동적 콘텐츠로 로드하는 경우, 기본 클릭 메소드가 작동되지 않음
            # 자바스크립트를 사용하여 클릭 이벤트 실행
            driver.execute_script("arguments[0].click();", button)

            break
    # 페이지가 로드되기 전에 대기
    wait = WebDriverWait(driver, 20)
    # 페이지의 첫 번째 링크 요소가 새로운 페이지 로딩으로 인해 사라질 때까지 대기
    wait.until(EC.staleness_of(page_bar[0]))
    # 새 페이지가 로드된 후, 특정 요소가 존재하는지 확인하여 페이지 로딩 완료를 확인
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/div[2]/div[1]/ul/li[1]/div[2]/div/a')))
    # 페이지 로딩 후 잠시 대기
    time.sleep(1.5)

# 한 면에서 넘길 수 있는 페이지 수는 5개
# 그 이후에는 다음 버튼을 클릭해야함.
# 다음 버튼 클릭 함수
def click_next_button():
    # CSS 셀렉터.btn_next.ico를 사용하여 다음 버튼을 찾음
    # CSS 셀렉터는 <a> 태그의 class 속성이 btn_next와 ico를 가진 요소를 찾음
    next_button = driver.find_element(By.CSS_SELECTOR, '.btn_next.ico')
    # 자바스크립트를 사용하여 클릭 이벤트 실행
    driver.execute_script("arguments[0].click();", next_button)
    # 페이지 로드 대기
    wait = WebDriverWait(driver, 20)
    wait.until(EC.staleness_of(next_button))
    # 새로운 페이지 로드 대기:
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/div[2]/div[1]/ul/li[1]/div[2]/div/a')))
    time.sleep(1.5)

네이버_리뷰_크롤링_최종_배포용.ipynb

In [ ]:
def extract_data():
    try:
        tourist_spot = spot

        # 사용 가능한 '이런 점이 좋았어요' 요소 개수 파악
        # MHaAm : '이런 점이 좋았어요' 요소 CLASS NAME
        # 따라서 MHaAm 로 접근하여 각 요소 리스트에 담고 길이 구하기
        good_points_elements = driver.find_elements(By.CLASS_NAME, 'MHaAm')
        num_good_points = len(good_points_elements)

        # 첫번째 부터 마지막 요소번째 까지 반복
        for j in range(0, num_good_points + 1):
            try:
                # 좋은 점과 선택한 사람 수 추출
                good_point = driver.find_elements(By.CLASS_NAME, 't3JSf')[j].text # 이런 점이 좋았어요 - 이런점 추출
                selected_people = driver.find_elements(By.CLASS_NAME, 'CUoLy')[j].text # 이런 점이 좋았어요 - 투표 수 추출
                spot_attribute = driver.find_element(By.CLASS_NAME, 'lnJFt').text # 관광지 속성(네이버 지도 - 관광지명 옆 작은 카테고리명)

                # 추출된 요소들 리스트에 넣기 - 후반부 DataFrame 생성시 사용
                good_point_list.append(good_point)
                selected_people_list.append(selected_people)
                spot_attribute_list.append(spot_attribute)
                tourist_spot_list.append(tourist_spot)

            except Exception as e:
                print(f"항목 {j}은(는) 존재하지 않습니다. 오류: {e}")
                if 'driver' in locals():
                    driver.quit()
                return None

    except Exception as e:
        print(f"데이터 추출 중 오류 발생: {e}")

## 전처리 & ERD

In [ ]:
# DB pivot table 변환 함수

def making_pivot(df):
    pivot_df = df.pivot_table(
        index='Attraction',
        columns='Category_Map',
        values='Selected_People',
        aggfunc='sum',
        fill_value=0
    )

    first_values = df.groupby('Attraction').first()[['Participants', 'Address', 'Tag', 'Longitude', 'Latitude']]
    pivot_df = pivot_df.reset_index().merge(first_values, on='Attraction', how='left')

    return pivot_df

In [ ]:
# 실루엣 점수 구하는 함수

def silhouetteviz(n_cluster, x_features):
    kmeans = KMeans(n_clusters=n_cluster, random_state=42)
    y_labels = kmeans.fit_predict(x_features)
    silhouette_values = silhouette_samples(x_features, y_labels, metric='euclidean')

    # 초기화
    y_ax_lower, y_ax_upper = 0, 0
    y_ticks=[]

    for c in range(n_cluster):
        c_silhouette = silhouette_values[y_labels == c]
        c_silhouette.sort()
        y_ax_upper += len(c_silhouette)
        color = cm.jet(float(c)/n_cluster)
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette,
                 height=1.0, edgecolor='none', color=color)
        y_ticks.append((y_ax_lower + y_ax_upper) / 2.)
        y_ax_lower += len(c_silhouette)
    silhouetter_avg = np.mean(silhouette_values)
    # print(silhouetter_avg)

    # 그래프 꾸며주는 코드
    plt.axvline(silhouetter_avg, color='red', linestyle='--') # 실루엣 평균 점수 선 표시
    plt.title('Number of Cluster: '+str(n_cluster)+'\n'\
              + 'Silhouette_score: ' +str(round(silhouetter_avg, 3)))

    plt.yticks(y_ticks, range(n_cluster))
    plt.xticks([0,0.2,0.4,0.6,0.8,1.0])
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')
    plt.tight_layout()
    plt.show()

In [ ]:
# 군집 분석 후 시각화해주는 함수

def clusterScatter(n_cluster, x_features, att1, att2):
    c_colors = []
    kmeans = KMeans(n_clusters=n_cluster, random_state=42)
    y_labels = kmeans.fit_predict(x_features)

    for i in range(n_cluster):
        c_color = cm.jet(float(i)/n_cluster)
        c_colors.append(c_color)
        # 클러스터 색상 설정

        # cluster 내 데이터 분포를 표현, centroid 동그라미로 시각화
        plt.scatter(x_features[y_labels ==i, att1], x_features[y_labels ==i, att2],
                    marker ='o', color=c_color, edgecolor='black' , s=50,
                    label='cluster'+str(i)
                    )
        # 각 클러스터의 중심점(centroid) 별(*)로 표시
    for i in range(n_cluster):
        plt.scatter(kmeans.cluster_centers_[i,att1],kmeans.cluster_centers_[i,att2],
                    marker='*', color=c_colors[i], edgecolor='w', s=100)

    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
# 텍스트 전처리 함수

def make_newTagCols(df, nlp):
    # tag 분리한거 담는 컬럼 만들기
    df[['newTag', 'newTag_list', 'tag_word_N']] = None
    # 모든 태그 모음집
    total = ''
    # 모든 태그의 명사 추출 모음집
    N_list = []

    # '#' 분리해서 컬럼에 담기
    for i, tag in enumerate(df.Tag):
        tag = tag.split('#')[1:]
        df['newTag_list'][i] = tag

        words = ''
        for t in tag:
            words = words + t + ' '

        total = total + words + ' '
        df['newTag'][i] = words

        # 명사 추출해서 또 새로운 컬럼에 담기
        newTag_N = list(set(nlp.nouns(words)))
        N_list.append(newTag_N)
        df['tag_word_N'][i] = newTag_N

    # '#'제거 해시태그 한 리스트에 모은 것 : total
    # total에서 명사 추출한 결과
    total_nouns = nlp.nouns(total)

    return df, total, N_list, total_nouns

In [ ]:
# '관광' 카테고리 관광지의 속성별 총비율 구하는 함수

def cluster_ratio_tour(df, cluster_num):
    # 관광지별 속성 합계
    df['합계'] = df['만족도'] + df['매력도'] + df['편의']

    # 클러스터 그룹화
    for i in range(1, cluster_num+1):
        cond = df['class'] == i
        df_cond = df[cond]

        총합계 = df_cond['합계'].sum()
        df_cond['합계/총합계'] = df_cond['합계'] / 총합계 * 100

        # 관광지별 속성 비율
        df_cond['만족도 비율'] = df_cond['만족도'] / df_cond['합계']
        df_cond['매력도 비율'] = df_cond['매력도'] / df_cond['합계']
        df_cond['편의 비율'] = df_cond['편의'] / df_cond['합계']

        # 총 관광지를 고려한 관광지별 속성 비율
        df_cond['만족도 총비율'] = df_cond['합계/총합계'] * df_cond['만족도 비율']
        df_cond['매력도 총비율'] = df_cond['합계/총합계'] * df_cond['매력도 비율']
        df_cond['편의 총비율'] = df_cond['합계/총합계'] * df_cond['편의 비율']

        df_cond.to_csv(f'./ratio/df_tour_cond{i}.csv')

## 모델링

In [ ]:
# LDA 응집도 구하고 시각화하는 함수

def coherence_scores(corpus, dictionary, texts, min_topics, max_topics):

    coherence_scores = []

    for num_topics in range(min_topics, max_topics):
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, random_state=42)
        coherence = CoherenceModel(model=model, texts=texts, dictionary=dictionary)
        coherence_scores.append(coherence.get_coherence())

    x = [i for i in range(min_topics, max_topics)]

    plt.figure(figsize=(10,6))
    plt.plot(x, coherence_score)
    # x축 값 : 최소~최대 범위의 수
    # y축 값 : 반복문으로 추출한 coherence_score
    plt.title('Tour')
    plt.xlabel('Number of Topics')
    plt.ylabel('Coherence Scores')
    plt.show()

    return coherence_scores

In [ ]:
# topic 매핑 함수

def max_count(x, y):
    # 최빈 단어 유형 추출 후 새로운 컬럼에 추가
    x['Max'] = x.max(axis=1)
    x['Max_Column'] = x.idxmax(axis=1)

    # temp가 아닌 원래 데이터에 컬럼명 변경하여 값 넣기
    y['max'] = x['Max']

    # temp가 아닌 원래 데이터에 컬럼명 변경하여 값 넣기
    y['topic_type'] = x['Max_Column']

In [ ]:
# 웹구현 & 콘텐츠 기반 유사도 관련 함수



# 해시태그 맵핑 함수
def map_hashtags(hashtags, category):
    tour_hashtag_mapping = {
        '#분위기': '매력도', '#시설': '편의', '#활동': '매력도',
        '#접근성': '편의', '#서비스': '만족도', '#가격': '만족도'
    }
    food_hashtag_mapping = {
        '#음식의 맛과 질': '음식의 속성', '#음식의 다양성': '음식의 속성',
        '#시설 및 환경': '시설 및 분위기', '#분위기': '시설 및 분위기',
        '#청결도와 위생': '시설 및 분위기', '#접근성': '시설 및 분위기',
        '#특별한 목적': '시설 및 분위기', '#가격과 가치': '가격 및 서비스',
        '#서비스': '가격 및 서비스'
    }

    if category == '관광':
        hashtag_mapping = tour_hashtag_mapping
    else:
        hashtag_mapping = food_hashtag_mapping

    mapped_hashtags = set(hashtag_mapping.get(hashtag, '')
                          for hashtag in hashtags if hashtag in hashtag_mapping)

    return list(mapped_hashtags)  # 결과를 다시 리스트로 반환하여 순서가 있는 데이터 구조 제공



# 피봇테이블 출력 함수
def making_pivot(category, popularity_class, types, hashtags, all_data):
    mapped_hashtags = map_hashtags(hashtags, category)

    # 데이터 필터링
    filtered_data = all_data[(all_data["Category"] == category) &
                             (all_data["class"] == popularity_class) &
                             (all_data["type"].isin(types)) &
                             (all_data["Category_Map"].isin(mapped_hashtags))]

    if not filtered_data.empty:
        pivot_table = filtered_data.pivot_table(
            index='Attraction', columns='Category_Map', values='Selected_People', aggfunc='sum', fill_value=0)

    if category == '관광':
        pivot_table = pd.merge(pivot_table, tour_ratio)
    else:
        pivot_table = pd.merge(pivot_table, food_ratio)
        return pivot_table.drop_duplicates()



# 대분류 피봇 테이블 출력 함수
def making_pivot_min(df, category):
    if category == '관광':
        df = df[df['Category'] == '관광']
    elif category == '음식':
        df = df[df['Category'] == '음식']

    return df.pivot_table(index='Attraction', columns='Category_Map', values='Selected_People', aggfunc='sum', fill_value=0).reset_index()



# 유사도 함수
def cosine_similarity_func(df, df2, category, types, attraction, top_n=5):

    condition = (df['Category'] == category) & (df['type'].isin(types))
    df_con = df[condition]
    temp = making_pivot_min(df_con, category)
    if category == '관광':
        feature_data = pd.merge(temp, df2).iloc[:, 7:10].values
    else:
        feature_data = pd.merge(temp, df2).iloc[:, 10:13].values
    cosine_sim = cosine_similarity(feature_data)
    index = temp[temp['Attraction'] == attraction].index[0]
    cosine_sim[index, index] = -np.inf
    most_similar_indices = np.argsort(cosine_sim[index])[-top_n:]
    recommendations = temp.iloc[most_similar_indices]['Attraction'].values.tolist(
    )
    return recommendations




# 랜덤 테마여행
def random_tour(df, topic):
    random.seed(42)
    tp1 = df[df['topic_type'] == topic]
    tp1_lst = tp1[tp1['max'] >= 0.95]['Attraction'].values
    shuffle(tp1_lst)
    tp1_lst = tp1_lst[:5].tolist()
    return tp1_lst